In [1]:
import numpy as np
import pandas as pd
import math
import pymatgen as mg
from pymatgen.transformations.standard_transformations import RemoveSpeciesTransformation
from pymatgen.transformations.standard_transformations import SupercellTransformation

In [2]:
structure = mg.Structure.from_file("cif/0D/2208808.cif")

cell = [structure.lattice.a, structure.lattice.b, structure.lattice.c]
print(cell)

u = SupercellTransformation(scaling_matrix=((2, 0, 0), (0, 2, 0), (0, 0, 2)))
supercell = u.apply_transformation(structure)

t = RemoveSpeciesTransformation(["H", "C", "N", "S", "O", "I"])
modified_structure = t.apply_transformation(supercell)

print(len(modified_structure))

# Pb-Pb distances in a table

dumb = np.array([modified_structure.get_distance(0, i) for i in range(1, len(modified_structure))])
# around
dumb1 = np.around(dumb,decimals=2)
# remove duplicates
dumb2 = np.array(list(set(dumb1)))
# sort values
dist_lst = np.sort(dumb2)
print(len(dist_lst))
print(dist_lst)

[15.093000000000002, 9.297, 19.142]
32
15
[ 8.86  9.3  10.47 10.64 13.7  14.   15.09 16.22 16.55 17.73 19.14 19.16
 20.93 21.28 22.91]


In [19]:
# Find d_min = shortest Pb-Pb bond
d_min = min(dist_lst)
print(d_min)

# Crop higher distance list
d_max = round(min(cell),2)
print(d_max)

dist_lst_min = np.delete(dist_lst, np.argwhere((dist_lst > d_max)))
print(dist_lst_min)

# Crop higher distances list
#d_max = 11
#dist_lst_max = np.delete(dist_lst_min, np.argwhere((dist_lst_min > d_max)))
#print(dist_lst_max)

8.86
9.3
[8.86 9.3 ]


In [20]:
# Volume approximated by cube
a = np.mean(dist_lst_min)
print(a)
vol_cube = round(a*a*a, 2 )
print("Cube volume", vol_cube, "A3")
vol_empty = round(vol_cube/2 , 2 )
print("Empty volume", vol_empty, "A3")

9.08
Cube volume 748.61 A3
Empty volume 374.3 A3


19.3509


In [22]:
# Remove distances due to small structural distortions   
disto = 0.1 #Angstrom
def distortions(dist_lst):
    for i in range(0, len(dist_lst)-1):
        a = round(dist_lst[i], 2)
        b = round(dist_lst[i+1], 2)
        if math.isclose(a, b, abs_tol = disto):
            dist_lst[i+1] = dist_lst[i]
    return dist_lst

In [23]:
dst_lst_new = distortions(dist_lst)
print(dst_lst_new)

[ 9.36  9.36 10.06 10.06 13.09 14.3  16.17 16.17 16.8  17.38 19.35 19.35
 22.03 23.36 25.63]
